### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd
from pathlib import Path
import win32com.client as win32

In [2]:
# importando as tabelas
emails_df = pd.read_excel(r'Bases de Dados/Emails.xlsx')
lojas_df = pd.read_csv(r'Bases de Dados/Lojas.csv', sep=';', encoding='latin1')
vendas_df = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

In [3]:
tabela_df = pd.merge(vendas_df, lojas_df, on='ID Loja')
display(tabela_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [4]:
dicionarios_lojas = {}

for loja in lojas_df['Loja']:
    filtro = tabela_df['Loja'] == loja
    tabela_filtrada = tabela_df.loc[filtro]
    dicionarios_lojas[loja] = tabela_filtrada

# filtrando uma das chaves do dicionario de lojas
display(dicionarios_lojas['Salvador Shopping'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
96969,81,2019-01-02,8,Cueca Xadrez,1,60,60,Salvador Shopping
96970,126,2019-01-02,8,Camisa Estampa,1,113,113,Salvador Shopping
96971,138,2019-01-02,8,Pulseira Linho,5,114,570,Salvador Shopping
96972,177,2019-01-02,8,Camisa Liso,2,105,210,Salvador Shopping
96973,194,2019-01-02,8,Camisa Gola V Linho,4,145,580,Salvador Shopping
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [5]:
# Indicador do dia = Ultimo dia disponivel na planilha (2019-12-26)
indicador = tabela_df['Data'].max()
print(indicador)

# Indicador do ano = 2019
print(indicador.year)

2019-12-26 00:00:00
2019


### Passo 3 - Salvar a planilha na pasta de backup

In [6]:
# Identificar se a pasta já existe
CAMINHO_BACKUP = Path(r"Backup Arquivos Lojas")

for loja in lojas_df['Loja']:
    if not Path(CAMINHO_BACKUP/ f'{loja}').exists():
        Path(CAMINHO_BACKUP/ f'{loja}').mkdir(parents=True, exist_ok=True)

# Salvar um arquivo excel dentro da pasta de cada loja
for loja, tabela in dicionarios_lojas.items():
    caminho = Path(CAMINHO_BACKUP) / loja
    loja = loja.replace(' ', '_')
    if not Path(caminho / f'{indicador.day}_{indicador.month}_{loja}.xlsx').exists():
        tabela.to_excel(Path(caminho) / f'{indicador.day}_{indicador.month}_{loja}.xlsx', index=False)

### Passo 4 - Calcular o indicador das lojas

### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

In [7]:
def definir_cor(valor, meta):
    return 'green' if valor >= meta else 'red'

# Dicionarios dos indicadores
dicionarios_faturamento = {}
dicionarios_produtos = {}
dicionarios_ticket_medio = {}

# Definicação das constantes de metas
META_FAT_DIARIA = 1_000
META_FAT_ANUAL = 1_650_000
META_PRODUTOS_DIA = 4
META_PRODUTOS_ANUAL = 120
META_TICKET_DIA = META_TICKET_ANUAL = 500

# Calculo dos indicadores
for loja, tabela in dicionarios_lojas.items():
    loja_dia = tabela.loc[tabela['Data'] == indicador, :]

    # Faturamento
    faturamento_ano = tabela['Valor Final'].sum()
    faturamento_dia = loja_dia['Valor Final'].sum()
    dicionarios_faturamento[loja] = {'Dia': faturamento_dia, 'Ano': faturamento_ano}

    # Diversidade de produtos# 
    produtos_loja_anual = tabela['Produto'].unique()
    produtos_loja_dia = loja_dia['Produto'].unique()
    dicionarios_produtos[loja] = {'Dia': len(produtos_loja_dia), 'Ano': len(produtos_loja_anual)}

    # Ticket Médio
    tabela_vendas_ano = tabela[['Código Venda', 'Valor Final']].groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_ano = tabela_vendas_ano['Valor Final'].mean()
    tabela_vendas_dia = loja_dia[['Código Venda', 'Valor Final']].groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = tabela_vendas_dia['Valor Final'].mean()
    dicionarios_ticket_medio[loja] = {'Dia': ticket_medio_dia if not 'nan' else 0, 'Ano': ticket_medio_ano}

    # Montando e-mail
    gerente = emails_df.loc[emails_df['Loja'] == loja, 'Gerente'].values[0]
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = emails_df.loc[emails_df['Loja'] == loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage {indicador.day}/{indicador.month}/{indicador.year} - Loja {loja}'

    # Aplicando a função para cada caso
    cor_fat_dia = definir_cor(dicionarios_faturamento[loja]['Dia'], META_FAT_DIARIA)
    cor_fat_ano = definir_cor(dicionarios_faturamento[loja]['Ano'], META_FAT_ANUAL)
    cor_prod_dia = definir_cor(dicionarios_produtos[loja]['Dia'], META_PRODUTOS_DIA)
    cor_prod_ano = definir_cor(dicionarios_produtos[loja]['Ano'], META_PRODUTOS_ANUAL)
    cor_ticket_dia = definir_cor(dicionarios_ticket_medio[loja]['Dia'], META_TICKET_DIA)
    cor_ticket_ano = definir_cor(dicionarios_ticket_medio[loja]['Ano'], META_TICKET_ANUAL)

    # vamos fazer em html
    mail.HTMLBody = f"""
    <p>Prezado, {gerente}</p>

    <p>O resultado de ontem <strong>({indicador.day}/{indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {dicionarios_faturamento[loja]['Dia']:,.2f}</td>
        <td style="text-align: center">R$ {META_FAT_DIARIA:,.2f}</td>
        <td style="text-align: center"><font color='{cor_fat_dia}'>◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{dicionarios_produtos[loja]['Dia']}</td>
        <td style="text-align: center">{META_PRODUTOS_DIA}</td>
        <td style="text-align: center"><font color='{cor_prod_dia}'>◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {dicionarios_ticket_medio[loja]['Dia']:,.2f}</td>
        <td style="text-align: center">R$ {META_TICKET_DIA:,.2f}</td>
        <td style="text-align: center"><font color='{cor_ticket_dia}'>◙</font></td>
    </tr>
    </table>
    <p></p>
    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {dicionarios_faturamento[loja]['Ano']:,.2f}</td>
        <td style="text-align: center">R$ {META_FAT_ANUAL:,.2f}</td>
        <td style="text-align: center"><font color='{cor_fat_ano}'>◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{dicionarios_produtos[loja]['Ano']}</td>
        <td style="text-align: center">{META_PRODUTOS_ANUAL}</td>
        <td style="text-align: center"><font color='{cor_fat_ano}'>◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {dicionarios_ticket_medio[loja]['Ano']:,.2f}</td>
        <td style="text-align: center">R$ {META_TICKET_ANUAL:,.2f}</td>
        <td style="text-align: center"><font color='{cor_ticket_ano}'>◙</font></td>
    </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
    <p>Qualquer duvida estou a disposição</p>
    <p>Atenciosamente, Túlio Rocha</p>    
    """

    loja_arquivo = loja.replace(' ', '_')
    attachment  = Path.cwd() / CAMINHO_BACKUP / loja / f'{indicador.day}_{indicador.month}_{loja_arquivo}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()

### Passo 7 - Criar ranking para diretoria

In [13]:
# Criando ranking anual
ranking_df = tabela_df[['Loja', 'Valor Final']].groupby('Loja').sum(numeric_only=True)
ranking_df = ranking_df.sort_values(by='Valor Final', ascending=False)
ranking_df = ranking_df.rename(columns={'Valor Final': 'Faturamento Anual'})
ranking_df['Faturamento Anual'] = ranking_df['Faturamento Anual'].apply(lambda x: f'R$ {x:,.2f}')
display(ranking_df)

,Faturamento Anual
Loja,
Iguatemi Campinas,"R$ 1,762,419.00"
Shopping Vila Velha,"R$ 1,731,167.00"
Bourbon Shopping SP,"R$ 1,726,110.00"
Rio Mar Recife,"R$ 1,722,766.00"
Shopping SP Market,"R$ 1,721,763.00"
Palladium Shopping Curitiba,"R$ 1,721,120.00"
Norte Shopping,"R$ 1,711,968.00"
Ribeirão Shopping,"R$ 1,707,122.00"
Iguatemi Esplanada,"R$ 1,699,681.00"


In [18]:
# Criando ranking diário
tabela_dia = tabela_df.loc[tabela_df['Data'] == indicador, :]
ranking_dia_df = tabela_dia[['Loja', 'Valor Final']].groupby('Loja').sum(numeric_only=True)
ranking_dia_df = ranking_dia_df.sort_values(by='Valor Final', ascending=False)
ranking_dia_df = ranking_dia_df.rename(columns={'Valor Final': 'Faturamento Diário'})
ranking_dia_df['Faturamento Diário'] = ranking_dia_df['Faturamento Diário'].apply(lambda x: f'R$ {x:,.2f}')
display(ranking_dia_df)

,Faturamento Diário
Loja,
Salvador Shopping,"R$ 3,950.00"
Novo Shopping Ribeirão Preto,"R$ 3,400.00"
Center Shopping Uberlândia,"R$ 2,651.00"
Shopping Eldorado,"R$ 2,391.00"
Shopping Center Interlagos,"R$ 1,582.00"
Shopping Recife,"R$ 1,366.00"
Norte Shopping,"R$ 1,259.00"
Shopping União de Osasco,"R$ 1,207.00"
Shopping Vila Velha,R$ 937.00


In [20]:
# criando arquivo excel para as tabelas faturamento
nome_arquivo_anual = f'{indicador.day}_{indicador.month}_Ranking_Anual.xlsx'
nome_arquivo_diario = f'{indicador.day}_{indicador.month}_Ranking_Diario.xlsx'

if not Path(CAMINHO_BACKUP / nome_arquivo_anual).exists():
    ranking_df.to_excel(Path(CAMINHO_BACKUP / nome_arquivo_anual), index=True)

if not Path(CAMINHO_BACKUP / nome_arquivo_diario).exists():
    ranking_dia_df.to_excel(Path(CAMINHO_BACKUP / nome_arquivo_diario), index=True)

### Passo 8 - Enviar e-mail para diretoria

In [29]:
# Montando e-mail para diretoria
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = emails_df.loc[emails_df['Loja'] == 'Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking das Lojas - Dia {indicador.day}/{indicador.month}/{indicador.year}'

mail.Body = f"""
Prezados, 

Segue em anexo arquivos com os rankings diários e anuais de todas as lojas.
Abaixo, algumas informações:

Maior faturamento do dia: Loja {ranking_dia_df.index[0]} com {ranking_dia_df.iloc[0, 0]} em faturamento.
Menor faturamento do dia: Loja {ranking_dia_df.index[-1]} com {ranking_dia_df.iloc[-1, 0]} em faturamento.

Maior faturamento do ano: Loja {ranking_df.index[0]} com {ranking_df.iloc[0, 0]} em faturamento.
Menor faturamento do ano: Loja {ranking_df.index[-1]} com {ranking_df.iloc[-1, 0]} em faturamento.

Para qualquer dúvida, estou a disposição.
Atenciosamente, Túlio Rocha
"""

attachment  = Path.cwd() / CAMINHO_BACKUP / nome_arquivo_diario
mail.Attachments.Add(str(attachment))

attachment  = Path.cwd() / CAMINHO_BACKUP / nome_arquivo_anual
mail.Attachments.Add(str(attachment))

mail.Send()